<p style="font-weight:bold;"> <span style="font-size: 36px"> Technical Margin Specifications </span> </p>

The aim of this notebook is to document the Technical Margin (TM) and its allocation to the Contractual Service Margin (CSM), Loss Component (LC), or Loss Recovery Component (LoReCo). The process is pictorially explained in the following flowchart. 

<a name="TOC"></a>
<center><img src="../Images/SpecificationsCSM.PNG" width="45%" style="float:center;">

Throughout this notebook we adopt the following variable notation:  we use the words *computed* vs. *expected* to differentiate the results provided by the application vs. this test notebook, respectively.

[Setup IFRS17 Calculation Engine](../Import/ImportScopeCalculation):

In [1]:
#!import "SpecificationsSetup"

---

# Import Data

<a id='year-and-period'></a>
## Current Period

Please select the target period (i.e. year and month) and Reporting Node based on the imported data:

In [1]:
var year          = 2021   ;
var month         = 3      ;
var reportingNode = "CH"   ;
var economicBasis = "L"    ;

Import the Dimensions, Parameters, Nominal Cashflows, and Actuals from the corresponding csv files in <code>Initialization/Systemorph/</code>

In [1]:
#!eval-notebook "../Initialization/InitSystemorphToMemory"

Initialization of the Test Suite: the data loaded from csv files are ready to be used by the Scopes for the calculations

In [1]:
var Test = await StartCalculatorAsync(reportingNode, year, month, economicBasis, Periodicity.Quarterly, ImportFormats.Cashflow);

<a id='view-loaded-data-nodes'></a>
## View loaded Data Nodes

The imported active Data Nodes from <code>Initialization/Systemorph/DataNodes.csv</code> are 

In [1]:
Test.GetStorage().DataNodeDataBySystemName

whereas the Data Nodes of your imported Cashflows are

In [1]:
Test.GetStorage().DataNodesByImportScope[ImportScope.Primary]

Please select here below the target Data Node for executing the tests below:

In [1]:
var dataNode = "DT1.1";

## View list of Identities

Based on the Cashflows imported, the list of all possible identities, i.e. combinations of valid Data Nodes, Aoc steps, and Novelties, is contructed and reported here below:

In [1]:
var allIdentities = Test.GetScopes<GetIdentities>(Test.GetStorage().DataNodesByImportScope[ImportScope.Primary]).SelectMany(s => s.Identities).ToArray();

In [1]:
allIdentities

<br/><br/>
<a id='present-value'></a>
# Present Value

Projected Present Values (PPVs) are calculated for each Amount Type (AT) either with the *locked-in* (L) and with the *current* (C) Yield Curves. While the former is the latest available as per end of the DataNode's inception year, the latter is the latest available as per the current period. 
PPVs for both Economic Basis L and C have been defined [here](../Import/ImportScopeCalculation#present-value) and tested [here](SpecificationsImportCashflows#present-values). 

According to the BBA methodology, the CSM calculation uses the PV with locked-in Yield Curve simply defined as

$$
\text{PV Locked}(\text{AoC}, \text{AT}) = \text{PPV}(\text{AoC}, \text{AT}) \big|_{\substack{ \text{Calculation Type = BE} \\ \text{Economic Base = L} } }
$$

In [1]:
var pvsLocked = allIdentities.SelectMany(id => Test.GetScope<PvLocked>(id).PresentValues
                                                   .Where(x => Math.Abs(x.Value) >= Precision)
                                                   .Select(x => x.FromPvToIfrsVariable())).ToArray();

Below a view of the computed locked-in PV 

In [1]:
Report.ForObjects(pvsLocked)
      .WithQuerySource(Workspace)
      .GroupColumnsBy(x => x.EconomicBasis)
      .GroupColumnsBy(x => x.AmountType)
      .GroupRowsBy(x => x.Novelty)
      .GroupRowsBy(x => x.AocType)
      .WithGridOptionsForIfrsVariable()
      .ToReport()

Aggregated values are also available as the sum over all [Amount Types](../DataModel/DataStructure#amount-type):

$$
\text{PV Locked Value}(\text{AoC}) = \sum_{\text{AT}} \text{PV Locked}(\text{AoC}, \text{AT})
$$

Here below an example

In [1]:
var id_BoP_I = Test.GetIdentity(dataNode, "BOP", "I");

In [1]:
id_BoP_I

In [1]:
Test.GetScope<PvLocked>( id_BoP_I ).Value

<br/><br/>
<a id='risk-adjustment'></a>
# Risk Adjustment

The CSM calculation also uses locked-in Risk Adjustment (RA Locked) which is retrieved from the corresponding Present Value with [Calculation Type](../DataModel/DataStructure#calculation-type) being RA defined [here](../Import/ImportScopeCalculation#present-value) and tested [here](SpecificationsImportCashflows#present-values). 

Risk Adjustment values can be written as

$$
\text{RA Locked Value}(\text{AoC}) = \text{PV}(\text{AoC})|_{\substack{ \text{Calculation Type = RA} \\ \text{Economic Base = L} }}
$$

where the imported Cashflows for the RA Calculation Type are already aggregated over all [Amount Types](../DataModel/DataStructure#amount-type).

Here below an example

In [1]:
var id_BoP_I = Test.GetIdentity(dataNode, "BOP", "I");

In [1]:
id_BoP_I

In [1]:
Test.GetScope<RaLocked>( id_BoP_I ).Value

The RA Locked per Data Node imported are

In [1]:
var rasLocked = allIdentities.SelectMany(id => Test.GetScope<RaLocked>(id).PresentValues
                                                   .Where(x => Math.Abs(x.Value) >= Precision)
                                                   .Select(x => x.FromPvToIfrsVariable())).ToArray();

In [1]:
Report.ForObjects<IfrsVariable>(rasLocked)
      .WithQuerySource(Workspace)
      .GroupColumnsBy(x => x.EconomicBasis)
      .GroupColumnsBy(x => x.DataNode)
      .GroupRowsBy(x => x.Novelty)
      .GroupRowsBy(x => x.AocType)
      .WithGridOptionsForIfrsVariable()
      .ToReport()

<br/><br/>
<a name='technical-margins'></a>
# Technical Margin

For the computation of the CSM or LC components for each AoC step, it is convenient to introduce the notion of technical margin (TM).

Firstly, it is useful to define the so-called Aggregated Technical Margin (ATM)

$$
\text{ATM} (\text{AoC}) = \sum_{s\in\text{previous AoC steps}} \text{TM}(s) ~,
$$

and the Interest Accretion Factor (IAF) 

$$
\text{IAF} = \prod_{i=1}^{\text{TS}} (1 + \text{YC}_i) - 1 ~.
$$

The TM is defined as

$$
\text{TM}(\text{AoC}) = \left\{
\begin{array}{rl}
\text{TM}(\rm{EOP}) \text{ of the previous period}                           ~ & \text{if }s = \text{BOP and Novelty is In-Force}. \\
0                                                                            ~ & \text{if }s = \text{CF}. \\
\text{IAF} \cdot ~ \text{ATM}(\text{AoC})                                    ~ & \text{if }s = \text{IA}. \\
\rm{Premiums} + \text{Attributable Expense and Commissions} + \text{Investment Claims}                    ~ & \text{if }s = \text{EA}. \\
-\text{AMF} \cdot ~ \text{ATM}(\text{AoC})                                   ~ & \text{if }s = \text{AM}. \\
\text{PV Locked Value}(\text{AoC}) 
\bigg|_{\substack{\text{Non Attributable} \\ 
                  \text{Amount Types} \\ 
                  \text{excluded}}} 
+ \text{RA Locked Value}(\text{AoC})                                         ~ & \text{otherwise} 
\end{array}
\right.
$$

where AMF is the Amortization Factor that is defined [here](../Import/ImportScopeCalculation#amortization-factor) and tested [here](SpecificationsImportCashflows#amortization-factor), with YC being the [Yield Curve](#yield-curves) and TS is the Time-Step, 
and the Premiums, Attributable Expense and Commissions and Investment Claims terms are given by:

$$
\begin{array}{lcl}
\rm{Premiums} &=& \text{Premium Allocation Factor} \cdot \sum_{\text{Amount Type}\in\{\text{PR and its children}\}}
                  \big(\text{PV}_{\text{Novelty = I}} + \text{PV}_{\text{Novelty = N}} \big) - 
                  \big(\text{Actual}_{\text{Novelty=C}} + \text{Advance Actual}_{\text{Novelty=C}} + \text{Overdue Actual}_{\text{Novelty=C}} \big) ~, \\
\text{Attributable Expense and Commissions} &=& \sum_{\text{Amount Type}\in\{\rm{ACA}, \rm{AEA}\}}
                   \big(\text{PV}_{\text{Novelty = I}} + \text{PV}_{\text{Novelty = N}} \big) - \text{Actual}_{\text{Novelty=C}} ~, \\
\text{Investment Claims } &=& \sum_{\text{Amount Type}\in\{\text{ICO and its children}\}}
                              \big(\text{PV}_{\text{Novelty = I}} + \text{PV}_{\text{Novelty = N}} \big) - 
                              \big( \text{Actual}_{\text{Novelty=C}} + \text{Advance Actual}_{\text{Novelty=C}} + \text{Overdue Actual}_{\text{Novelty=C}} \big) ~.
\end{array}
$$

The AoC Type **CF** is implicit for all formulas, PV is the [present value](#present-value) with Calculation Type **BE**, and Actuals have Estimate Types **A**, **OA** and **AA** (see details [here](../Import/ImportScopeCalculation#actual-base)).

## Beginning of Period

In [1]:
var id_BoP_I = Test.GetIdentity(dataNode, "BOP", "I");

In [1]:
id_BoP_I

In [1]:
var computedTM_BoP_I = Test.GetScope<TechnicalMargin>(id_BoP_I);

In [1]:
computedTM_BoP_I.Value

The TM for BOP is equal to the TM for EOP of the previous period, that is, the sum of Csm, Loss Component and LoReCo as of at the end of last period (see the corresponding [formula](#technical-margins)). 

The corresponding **IFRS Variables** can be loaded directly from the *Storage*

In [1]:
var valueCsm = Test.GetStorage().GetValue(id_BoP_I, null, EstimateTypes.C, null);
valueCsm

In [1]:
var valueLc = Test.GetStorage().GetValue(id_BoP_I, null, EstimateTypes.L, null);
valueLc

In [1]:
var valueLr = Test.GetStorage().GetValue(id_BoP_I, null, EstimateTypes.LR, null);
valueLr

In [1]:
var expectedTM_BoP_I = -valueCsm + valueLc + valueLr;

In [1]:
expectedTM_BoP_I

In [1]:
computedTM_BoP_I.Value.Should().BeApproximately( expectedTM_BoP_I, Precision );

<a id='model-corrections'></a>
## Model Corrections

In [1]:
var id_MC_I = Test.GetIdentity(dataNode, "MC", "I");

In [1]:
id_MC_I

In [1]:
var computedTM_MC_I = Test.GetScope<TechnicalMargin>(id_MC_I);

In [1]:
computedTM_MC_I.Value

The TM for MC is given by the default [formula](#technical-margins), i.e. it is equal to the sum of PV and RA Locked, where the former does not encompass the *NonAttributable* Amount Types

In [1]:
var expectedTM_MC_I = Test.GetScope<PvLocked>(id_MC_I).Value + Test.GetScope<RaLocked>( id_MC_I ).Value;

In [1]:
expectedTM_MC_I

In [1]:
computedTM_MC_I.Value.Should().BeApproximately( expectedTM_MC_I, Precision );

## Cash Flow

In [1]:
var id_CF_I = Test.GetIdentity(dataNode, "CF", "I");

In [1]:
id_CF_I

In [1]:
var computedTM_CF_I = Test.GetScope<TechnicalMargin>(id_CF_I);

In [1]:
computedTM_CF_I.Value

According to the [formula](#technical-margins) above, the tecnical margin is 0 for the Cash Flow AoC step 

In [1]:
computedTM_CF_I.Value.Should().BeApproximately( 0, Precision );

<a id='interest-accretion'></a>
## Interest Accretion

In [1]:
var id_IA_I = Test.GetIdentity(dataNode, "IA", "I");

In [1]:
id_IA_I

In [1]:
var computedTM_IA_I = Test.GetScope<TechnicalMargin>(id_IA_I);

In [1]:
computedTM_IA_I.Value

According to the [formula](#technical-margins) above, for the AoC step IA, the TM is equal to sum of the TMs of its previous AoC step multiplied by the IAF. 

The IAF associated to the current period (i.e. shift and time step) can be calculated from the shift, the time Step, and the Interest Rates

In [1]:
var projectionPeriod = 0;

In [1]:
var shift = Test.GetStorage().GetShift(projectionPeriod);

In [1]:
var timeStep = Test.GetStorage().GetTimeStep(projectionPeriod);

In [1]:
(shift, timeStep)

In [1]:
var monthlyRates = Test.GetScope<MonthlyRate>(id_IA_I);

In [1]:
monthlyRates

In [1]:
var IAF = Enumerable.Range(shift,timeStep).Select(i => monthlyRates.Interest.GetValidElement(i/12)).Aggregate(1.0d, (x, y) => x * y ) - 1.0d;

In [1]:
IAF

Since the previous AoC steps are

In [1]:
var previousAocStep = Test.GetScope<PreviousAocSteps>((id_IA_I, InputSource.Cashflow)).Values;
previousAocStep

the expected TM for the Amortization step reads

In [1]:
var expectedTM_IA_I = IAF * previousAocStep.Sum(aoc => Test.GetScope<TechnicalMargin>(id_IA_I with {AocType = aoc.AocType, Novelty = aoc.Novelty}).Value);

In [1]:
expectedTM_IA_I

In [1]:
computedTM_IA_I.Value.Should().BeApproximately( expectedTM_IA_I, Precision );

## Combined Liabilities

In [1]:
var id_CL_C = Test.GetIdentity(dataNode, "CL", "C");

In [1]:
id_CL_C

In [1]:
var computedTM_CL_C = Test.GetScope<TechnicalMargin>(id_CL_C);

In [1]:
computedTM_CL_C.Value

According to the default TM [formula](#technical-margins), for the CL step the TM is equal to the sum of PV and RA Locked, where the former does not encompass the *NonAttributable* Amount Types

In [1]:
var expectedTM_CL_C = Test.GetScope<PvLocked>(id_CL_C).Value + Test.GetScope<RaLocked>( id_CL_C ).Value;

In [1]:
expectedTM_CL_C

In [1]:
computedTM_CL_C.Value.Should().BeApproximately( expectedTM_CL_C, Precision );

## Experience Adjustment

In [1]:
var id_EA_C = Test.GetIdentity(dataNode, "EA", "C");

In [1]:
id_EA_C

In [1]:
var computedTM_EA_C = Test.GetScope<TechnicalMargin>(id_EA_C);

In [1]:
computedTM_EA_C.Value

According to the [formula](#technical-margins) above, for the AoC step EA, the TM is equal to sum of the Premiums, Attributable Expense and Commissions, and Investment Claims. Here below we calculate these terms individually.

The Premiums, Attributable Expense and Commissions and Investment Claims Present Values and Actuals are calculated using the CF AoC step. Therefore, as first step we define the relevant AoC step and Novelties:

In [1]:
var referenceAoCType = Test.GetScope<ReferenceAocStep>(id_EA_C).Value.AocType;
var referenceAoC = Test.GetStorage().GetNovelties().Select(n => new AocStep(referenceAoCType, n));

In [1]:
referenceAoC

**Premiums**

The list of Premiums Amount Type defined in the imported dimensions and the Premium Allocation Factor can be retrieved

In [1]:
Test.GetStorage().GetPremiums()

In [1]:
var premiumAllocationFactor = Test.GetStorage().GetPremiumAllocationFactor(id_EA_C);

In [1]:
premiumAllocationFactor

Lastly, the Premiums term is

In [1]:
var premiums = premiumAllocationFactor * Test.GetStorage().GetPremiums()
    .Sum(p => referenceAoC.Sum(aoc => Test.GetScope<PresentValue>((Test.GetIdentity(dataNode, aoc.AocType, aoc.Novelty), p, EstimateTypes.BE, (int?)null), o => o.WithContext(economicBasis)).Value)
             -referenceAoC.Sum(aoc => Test.GetScope<ActualBase>((Test.GetIdentity(dataNode, aoc.AocType, aoc.Novelty), p, EstimateTypes.A, (int?)null)).Value)
             -referenceAoC.Sum(aoc => Test.GetScope<ActualBase>((Test.GetIdentity(dataNode, aoc.AocType, aoc.Novelty), p, EstimateTypes.AA, (int?)null)).Value)
             -referenceAoC.Sum(aoc => Test.GetScope<ActualBase>((Test.GetIdentity(dataNode, aoc.AocType, aoc.Novelty), p, EstimateTypes.OA, (int?)null)).Value)
        );

In [1]:
premiums

**Attributable Expense and Commissions**

Analogously, the Attributable Expense and Commissions term can be computed

In [1]:
var attributableExpenseAndCommissions = new string[] {AmountTypes.ACA, AmountTypes.AEA}
    .Sum(d => referenceAoC.Sum(s => Test.GetScope<PresentValue>((Test.GetIdentity(dataNode, s.AocType, s.Novelty), d, EstimateTypes.BE, (int?)null), o => o.WithContext(economicBasis)).Value)
             -referenceAoC.Sum(s => Test.GetScope<ActualBase>((Test.GetIdentity(dataNode, s.AocType, s.Novelty), d, EstimateTypes.A, (int?)null)).Value));

In [1]:
attributableExpenseAndCommissions

**Investment Claims**

Finally the Investment Claims term reads

In [1]:
var investmentClaims = Test.GetStorage().GetInvestmentClaims()
    .Sum(ic => referenceAoC.Sum(s => Test.GetScope<PresentValue>((Test.GetIdentity(dataNode, s.AocType, s.Novelty), ic, EstimateTypes.BE, (int?)null), o => o.WithContext(economicBasis)).Value)
              -referenceAoC.Sum(s => Test.GetScope<ActualBase>((Test.GetIdentity(dataNode, s.AocType, s.Novelty), ic, EstimateTypes.A, (int?)null)).Value)
              -referenceAoC.Sum(s => Test.GetScope<ActualBase>((Test.GetIdentity(dataNode, s.AocType, s.Novelty), ic, EstimateTypes.AA, (int?)null)).Value)
              -referenceAoC.Sum(s => Test.GetScope<ActualBase>((Test.GetIdentity(dataNode, s.AocType, s.Novelty), ic, EstimateTypes.OA, (int?)null)).Value)
        );

In [1]:
investmentClaims

**Expected TM**

In [1]:
var expectedTM_EA_C = premiums + attributableExpenseAndCommissions + investmentClaims;

In [1]:
expectedTM_EA_C

In [1]:
computedTM_EA_C.Value.Should().BeApproximately( expectedTM_EA_C, Precision );

## Amortization

In [1]:
var id_AM_C = Test.GetIdentity(dataNode, "AM", "C");

In [1]:
id_AM_C

In [1]:
var computedTM_AM_C = Test.GetScope<TechnicalMargin>(id_AM_C);

In [1]:
computedTM_AM_C.Value

In [1]:
computedTM_AM_C

According to the [formula](#technical-margins) above, for the AoC step AM, the TM is equal to sum of the TMs of its previous AoC step multiplied by the amortization factor with opposite sign. 

The AoC step previous to Amortization are

In [1]:
var previousAocStep = Test.GetScope<PreviousAocSteps>((id_AM_C, InputSource.Cashflow)).Values;
previousAocStep

while the Amortization Factor can be retrieved in the following

In [1]:
Test.GetScope<CurrentPeriodAmortizationFactor>(id_AM_C)

Therefore, the expected TM for the Amortization step reads

In [1]:
var expectedTM_AM_C = -Test.GetScope<CurrentPeriodAmortizationFactor>(id_AM_C).Value * 
                       previousAocStep.Sum(aoc => Test.GetScope<TechnicalMargin>(id_AM_C with {AocType = aoc.AocType, Novelty = aoc.Novelty}).Value);

In [1]:
expectedTM_AM_C

In [1]:
computedTM_AM_C.Value.Should().BeApproximately( expectedTM_AM_C, Precision );

## End of Period

In [1]:
var id_EoP_C = Test.GetIdentity(dataNode, "EOP", "C");

In [1]:
id_EoP_C

In [1]:
var computedTM_EoP_C = Test.GetScope<TechnicalMargin>(id_EoP_C);

In [1]:
computedTM_EoP_C.Value

In [1]:
var expectedTM_EoP_C = Test.GetScope<PvLocked>( id_EoP_C ).Value + Test.GetScope<RaLocked>( id_EoP_C ).Value;

In [1]:
expectedTM_EoP_C

In [1]:
computedTM_EoP_C.Value.Should().BeApproximately( expectedTM_EoP_C, Precision );

<br/><br/>
<a name='switch-logic'></a>
# Switch Logic    

The CSM and LC figures are allocated for each AoC step based on the sign of the ATM. Specifically, for positive (negative) ATM, the TM of the current step is allocated to the LC (CSM), unless the TM flips the sign of the ATM. This special circumstance is named **switch**. It can happen at any AoC step with the only exception of Amortization where there is no switch from the previous step.
When a switch occurs the total contribution to the CSM (LC) prior the switching step is brought to 0 and the remaing amount is allocated to LC (CSM).

In the following the AoC chain is investigated.

## Beginning of Period

For the BOP, the last period is considered, which dictates that the corresponding EOP figures considered have necessarily Novelty of type *In-Force*.

$$
\begin{array}{rcl}
\text{CSM}(\text{BOP}) &=& \text{CSM}(\text{EOP}) \text{ of the previous period, for Novelty In-Force} \\
\text{LC}(\text{BOP}) &=& \text{LC}(\text{EOP}) \text{ of the previous period, for Novelty In-Force}
\end{array}
$$

In [1]:
var id_BoP_I = Test.GetIdentity(dataNode, "BOP", "I");

In [1]:
id_BoP_I

In [1]:
var computedCSM_BoP_I = Test.GetScope<Csm>(id_BoP_I).Value;
var computedLC_BoP_I = Test.GetScope<Lc>(id_BoP_I).Value;

In [1]:
(computedCSM_BoP_I, computedLC_BoP_I)

To understand where the BOP is allocated, we retrieve the corresponding TM and, based on its sign, we expect the CSM or the LC to be non-zero according to the [formula](#switch-logic) above

In [1]:
var TM_BOP_I = Test.GetScope<TechnicalMargin>(id_BoP_I).Value;

In [1]:
TM_BOP_I

In [1]:
var expectedCSM_BoP_I = TM_BOP_I < 0 ? -TM_BOP_I : 0;
var expectedLC_BoP_I  = TM_BOP_I > 0 ? +TM_BOP_I : 0;

In [1]:
(expectedCSM_BoP_I, expectedLC_BoP_I)

In [1]:
computedCSM_BoP_I.CheckEquality(computedCSM_BoP_I).Should().Be(true);
expectedLC_BoP_I.CheckEquality(computedLC_BoP_I).Should().Be(true);

## Following AoC steps

From the BOP up to the CL step, the switch logic is applied separately to the In-Force and New Business novelties.

In [1]:
var id_MC_I = Test.GetIdentity(dataNode, "MC", "I");

In [1]:
id_MC_I

In [1]:
var computedCSM_MC_I = Test.GetScope<Csm>(id_MC_I).Value;
var computedLC_MC_I = Test.GetScope<Lc>(id_MC_I).Value;

In [1]:
(computedCSM_MC_I, computedLC_MC_I)

According to the [formula](#switch-logic) above this AoC step is allocated either to CSM or to LC based on the sign of ATM + TM. Both these two terms can be retrieved from the **TechnicalMargin** Scope.

In [1]:
var TM_MC_I = Test.GetScope<TechnicalMargin>(id_MC_I);

In [1]:
TM_MC_I

The correctness of the ATM can be checked by firstly withdrawing the previous AoC steps

In [1]:
var previousAocStep = Test.GetScope<PreviousAocSteps>((id_MC_I, InputSource.Cashflow)).Values;

In [1]:
previousAocStep

and then evaluating the sum of the corresponding TMs

In [1]:
var expectedATM = previousAocStep.Sum(aoc => Test.GetScope<TechnicalMargin>(id_MC_I with {AocType = aoc.AocType, Novelty = aoc.Novelty}).Value);

In [1]:
expectedATM

Therefore, the following condition for ATM calculated by the **TechnicalMargin** Scope needs to be fullfilled

In [1]:
TM_MC_I.AggregatedValue.Should().BeApproximately( expectedATM, Precision );

The CSM and LC can be allocated according to the [Switch Logic](#switch-logic).

In [1]:
double expectedCSM_MC_I; 
double expectedLC_MC_I; 

In [1]:
var aggregated = TM_MC_I.Value + TM_MC_I.AggregatedValue;

If there is no switch from LC, i.e. $\text{ATM} (\text{AoC}) > 0$ and $\text{ATM} (\text{AoC}) + \text{TM}(\text{AoC}) \ge 0 $, then:

$$
\begin{array}{rcl}
\text{CSM}(\text{AoC}) &=& 0 ~, \\
\text{LC}(\text{AoC}) &=& \text{TM}(\text{AoC}) ~.
\end{array}
$$

In [1]:
if(TM_MC_I.AggregatedValue > 0 && aggregated >= 0) {
    expectedCSM_MC_I = 0 ;
    expectedLC_MC_I = TM_MC_I.Value ;
} 

If there is no switch from CSM, i.e. $\text{ATM} (\text{AoC}) \le 0$ and $\text{ATM} (\text{AoC}) + \text{TM}(\text{AoC}) \le 0 $, then:

$$
\begin{array}{rcl}
\text{CSM}(\text{AoC}) &=& -\text{TM}(\text{AoC}) ~, \\
\text{LC}(\text{AoC}) &=& 0 ~.
\end{array}
$$

In [1]:
if(TM_MC_I.AggregatedValue < 0 && aggregated <= 0) {
    expectedCSM_MC_I = -TM_MC_I.Value ;
    expectedLC_MC_I = 0 ;
}

If $\text{ATM} (\text{AoC}) > 0$ and $\text{ATM} (\text{AoC}) + \text{TM}(\text{AoC}) \le 0$, then there is a switch to CSM:

$$
\begin{array}{rcl}
\text{CSM}(\text{AoC}) &=& -\text{TM}(\text{AoC}) -\text{ATM} (\text{AoC}) ~, \\
\text{LC}(\text{AoC}) &=& -\text{ATM} (\text{AoC}) ~.
\end{array}
$$

In [1]:
if(TM_MC_I.AggregatedValue > 0 && aggregated <= 0) {
    expectedCSM_MC_I = -aggregated;
    expectedLC_MC_I = -TM_MC_I.AggregatedValue ;
}

On the other hand, if $\text{ATM} (\text{AoC}) \le 0$ and $\text{ATM} (\text{AoC}) + \text{TM}(\text{AoC}) > 0$, 
then the switch is to LC:

$$
\begin{array}{rcl}
\text{CSM}(\text{AoC}) &=& \text{ATM} (\text{AoC}) ~, \\
\text{LC}(\text{AoC}) &=& \text{TM}(\text{AoC}) + \text{ATM} (\text{AoC}) ~.
\end{array}
$$

In [1]:
if(TM_MC_I.AggregatedValue <= 0 && aggregated > 0) {
    expectedCSM_MC_I = TM_MC_I.AggregatedValue ;
    expectedLC_MC_I = aggregated ;
}

Therefore, the expected CSM and LC for this step are

In [1]:
(expectedCSM_MC_I, expectedLC_MC_I)

In [1]:
expectedCSM_MC_I.CheckEquality(computedCSM_MC_I).Should().Be(true);
expectedLC_MC_I.CheckEquality(computedLC_MC_I).Should().Be(true);

<a name='combined-liabilities'></a>
## Combined Liabilities

The CL Aoc Step brings both contributions to CSM and LC together as the novelities are summed.

For the CL step, the logic is similar to the one above, except that a switch from LC to CSM can happen **because of New Business and despite of In-Force**, in which case we have:

$$
\begin{array}{rcl}
\text{CSM}(\text{AoC}) &=& -\text{TM}(\text{AoC}) - \text{ATM}(\text{Last In-Force AoC step}) - \text{TM}(\text{Last In-Force AoC step}) \\
\text{LC}(\text{AoC}) &=& -\text{ATM}(\text{Last In-Force AoC step}) - \text{TM}(\text{Last In-Force AoC step})
\end{array}
$$

If, on the other hand, the switch from LC to CSM happens **because of In-Force and despite of New Business**, then we have:

$$
\begin{array}{rcl}
\text{CSM}(\text{AoC}) &=& - \text{TM}(\text{AoC}) - \text{ATM}(\text{Last NB AoC step}) - \text{TM}(\text{Last NB AoC step}) \\
\text{LC}(\text{AoC}) &=& - \text{ATM}(\text{Last NB AoC step}) - \text{TM}(\text{Last NB AoC step})
\end{array}
$$

For the switch in the other direction, i.e. from CSM to LC the formulas are similar except that LC and CSM are swapped.

In [1]:
var id_CL_C = Test.GetIdentity(dataNode, "CL", "C");

In [1]:
id_CL_C

In [1]:
var computedCSM_CL_C = Test.GetScope<Csm>(id_CL_C).Value;
var computedLC_CL_C = Test.GetScope<Lc>(id_CL_C).Value;

In [1]:
(computedCSM_CL_C, computedLC_CL_C)

The CL step considers the last step of both the New Business and the In Force Novelties to decide what to allocate to CSM or LC. 

In [1]:
var lastAocSteps = Test.GetScope<PreviousAocSteps>((id_CL_C, InputSource.Cashflow)).Values.GroupBy(x => x.Novelty).Select(g => g.Last());

In [1]:
lastAocSteps

Selecting the corresponding AocTypes:

In [1]:
var lastAocTypeNb = lastAocSteps.Single(x => x.Novelty == Novelties.N).AocType;
var lastAocTypeI = lastAocSteps.Single(x => x.Novelty == Novelties.I).AocType;
(lastAocTypeI, lastAocTypeNb)

In [1]:
var lastTM_NewBusiness = Test.GetScope<TechnicalMargin>(id_CL_C with {AocType = lastAocTypeNb, Novelty = Novelties.N});

In [1]:
lastTM_NewBusiness

In [1]:
var lastTM_InForce = Test.GetScope<TechnicalMargin>(id_CL_C with {AocType = lastAocTypeI, Novelty = Novelties.I});

In [1]:
lastTM_InForce

Conversely the TM and ATM for the CL step is

In [1]:
var TM_Combined = Test.GetScope<TechnicalMargin>(id_CL_C);

In [1]:
TM_Combined

where the ATM of CL is given by the sum of the ATMs of the New Business and In Force Novelties

In [1]:
var aggregatedNewBusiness =  lastTM_NewBusiness.Value + lastTM_NewBusiness.AggregatedValue;
var aggregatedInForce = lastTM_InForce.Value + lastTM_InForce.AggregatedValue;
var sum = aggregatedNewBusiness + aggregatedInForce;
sum

In [1]:
( (sum - TM_Combined.AggregatedValue)/sum < 1.0e-12).Should().Be(true);

In order to decide where the TM of the CL step is allocated, one has to check the sign of the ATM + TM for both the New Business and the In Force

In [1]:
var aggregatedCombined = TM_Combined.Value + TM_Combined.AggregatedValue;
aggregatedCombined

According to the [formulas above](#combined-liabilities), for a positive (negative) aggregated CL value, the CL TM has to be allocated to LC (CSM). 
<br> In case the one of last Aoc step of the novelties (I, NB) has the sign of the CL ATM opposite to the CL ATM, the CL AocStep is used to introduce a balancing item to bring the ATM of the novelty with opposite sign to 0. 
<br> This is equivalent to execute the switch logic with a balancing item = ATM of the last AocStep having opposite sign to CL ATM. 
<br> Note that if CL TM is zero this results in allocating the same amount with opposite sign to CSM(note that the sign of the CSM componet is flipped) and LC.

In [1]:
double expectedCSM_CL_C; 
double expectedLC_CL_C; 

In [1]:
var balancingItem = aggregatedCombined < 0 ? aggregatedNewBusiness : aggregatedInForce;
balancingItem

In [1]:

if(aggregatedCombined > 0) {
    expectedCSM_CL_C = balancingItem ;
    expectedLC_CL_C = TM_Combined.Value + balancingItem;
} else {
    expectedLC_CL_C = - balancingItem ;
    expectedCSM_CL_C = TM_Combined.Value - balancingItem ;
}

In [1]:
(expectedCSM_CL_C, expectedLC_CL_C)

In [1]:
expectedCSM_CL_C.CheckEquality(computedCSM_CL_C).Should().Be(true);
expectedLC_CL_C.CheckEquality(computedLC_CL_C).Should().Be(true);

## End Of Period

For the last AoC step, the EOP is the sum of all previous steps

$$
\begin{array}{rcl}
\text{CSM}(\text{EOP}) &=& \sum_{s~\in~\text{previous AoC steps}} \text{CSM}(s) ~, \\
\text{LC}(\text{EOP}) &=& \sum_{s~\in~\text{previous AoC steps}} \text{LC}(s) ~.
\end{array}
$$

In [1]:
var id_EoP_C = Test.GetIdentity(dataNode, "EOP", "C");

In [1]:
id_EoP_C

In [1]:
var computedCSM_EoP_C = Test.GetScope<Csm>(id_EoP_C).Value;
var computedLC_EoP_C = Test.GetScope<Lc>(id_EoP_C).Value;

In [1]:
(computedCSM_EoP_C, computedLC_EoP_C)

<br/><br/>
<div class="alert alert-block alert-info">
For the <b/>reinsurance case</b>, the switch logic is identical to the one described above, except that it uses the corresponding gross case TM to allocate the figure either to CSM or to LoReCo. The reinsurance gross TM multiplied by the weights coming from the Reinsurance Coverage Parameter (Data Node Parameters). In case a GRIC has multiple GICs, then these weighted TMs are aggregated.
</div>

<a name="switch-logic"></a>

<br/><br/>
<a name='csm-lc'></a>
# Contractual Service Margin and Loss Component

Here below a view of the CSM and Loss Component results are shown. 

Note: the Data Nodes with LiabilityType being *LIC* are excluded.

In [1]:
var allIdentitiesWoLic = allIdentities.Where(id => Test.GetStorage().DataNodeDataBySystemName[id.DataNode].LiabilityType != "LIC").ToArray();

In [1]:
var csm = allIdentitiesWoLic.SelectMany(id => Test.GetScope<Csm>(id).RepeatOnce()
                                                  .Where(x => Math.Abs(x.Value) >= Precision)
                                                  .Select(x => x.FromCsmToIfrsVariable())).ToArray();

## Gross case

In [1]:
var allInsuranceIdentitiesWoLic = allIdentitiesWoLic.Where(id => !id.IsReinsurance).ToArray();

In [1]:
var lc = allInsuranceIdentitiesWoLic.SelectMany(id => Test.GetScope<Lc>(id).RepeatOnce()
                                                  .Where(x => Math.Abs(x.Value) >= Precision)
                                                  .Select(x => x.FromLcToIfrsVariable())).ToArray();

## Re-Insurance case (LoReCo)

In [1]:
var allReInsuranceIdentitiesWoLic = allIdentitiesWoLic.Where(id => id.IsReinsurance).ToArray();

In [1]:
var loreco = allReInsuranceIdentitiesWoLic.SelectMany(id => Test.GetScope<LoReCo>(id).RepeatOnce()
                                                     .Where(x => Math.Abs(x.Value) >= Precision)
                                                     .Select(x => x.FromLoReCoToIfrsVariable())).ToArray();

## Summary

In [1]:
Report.ForObjects( csm.Concat(lc).Concat(loreco) )
      .WithQuerySource(Workspace)
      .GroupColumnsBy(x => x.DataNode)
      .GroupColumnsBy(x => x.EstimateType)
      .GroupRowsBy(x => x.Novelty)
      .GroupRowsBy(x => x.AocType)
      .WithGridOptionsForIfrsVariable()
      .ToReport()